Example of querying a multivalued varchar column (e.g. `altids VARCHAR[]`)


In [12]:
import duckdb

cn = duckdb.connect()
cn.execute("create table test (pid VARCHAR, altids VARCHAR[] DEFAULT NULL)")
test_data = (
    ["pid_1", ["alt_1", "alt_2"],],
    ["pid_2", ["alt_3", ],],
    ["pid_3", ["alt_1", ],],
    ["pid_4", None],
)
for row in test_data:
    cn.execute("insert into test values (?, ?)", row)
cn.commit()


In [13]:
# Fetch all the rows at once (problematic with very large results)
result = cn.execute("select * from test")
for row in result.fetchall():
    print(row)

('pid_1', ['alt_1', 'alt_2'])
('pid_2', ['alt_3'])
('pid_3', ['alt_1'])
('pid_4', None)


In [14]:
# Fetch all the rows, but one at a time (use this pattern when there may be lots of results)
result = cn.execute("select * from test")
while row := result.fetchone():
    print(row)

('pid_1', ['alt_1', 'alt_2'])
('pid_2', ['alt_3'])
('pid_3', ['alt_1'])
('pid_4', None)


In [17]:
# Fetch records containing altid "alt_1"
result = cn.execute("select * from test where ? in altids", ("alt_1",))
for row in result.fetchall():
    print(row)


('pid_1', ['alt_1', 'alt_2'])
('pid_3', ['alt_1'])


In [18]:
# Same as above but using the list_contains() function
result = cn.execute("select * from test where list_contains(altids, ?)", ("alt_1",))
for row in result.fetchall():
    print(row)


('pid_1', ['alt_1', 'alt_2'])
('pid_3', ['alt_1'])


In [16]:
# No results returned altid = 'foo'
result = cn.execute("select * from test where list_contains(altids, ?)", ("foo",))
while row := result.fetchone():
    print(row)